In [7]:
import keras_ocr
import cv2

pipeline = keras_ocr.pipeline.Pipeline()
image_path = 'outputs/2024_1126_150000F_trimmed/object_detection/cropped_vessel_imgs/2024_1126_150000F_trimmed_cropped_vessel_id_1_frame_79.png'
image = cv2.imread(image_path).resize(300,300)

results = pipeline.recognize(image)

print(results)

In [ ]:
import keras_ocr
import cv2

# Initialize Keras-OCR pipeline
pipeline = keras_ocr.pipeline.Pipeline()

# Load and preprocess image
image = cv2.imread(image_path)
resized_image = cv2.resize(image, (800, 600))
grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
_, binary_image = cv2.threshold(grayscale_image, 128, 255, cv2.THRESH_BINARY)
processed_image = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2RGB)

# Adjust detection thresholds
pipeline.detector.detection_threshold = 0.8
pipeline.detector.text_threshold = 0.5
pipeline.detector.link_threshold = 0.5

# Perform OCR
results = pipeline.recognize([processed_image])

# Print detected text
for text, box in results[0]:
    print(f"Detected text: {text}")


Looking for C:\Users\dbacad\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dbacad\.keras-ocr\crnn_kurapan.h5


In [1]:
# TRY RETINA MASK ARGUMENT!
!nvidia-smi

Sat Jan 11 13:06:25 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 538.27                 Driver Version: 538.27       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 2000 Ada Gene...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P4              10W /  30W |      0MiB /  8188MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch

In [3]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)  # Should return '12.2'
print(torch.cuda.get_device_name(0))  # Should display your GPU name


True
12.1
NVIDIA RTX 2000 Ada Generation Laptop GPU


In [4]:
print(torch.backends.cudnn.is_available())  # Should return True
print(torch.backends.cudnn.version())  # Should return the installed cuDNN version


True
90100


### Import Libraries

In [5]:
import numpy as np
import cv2
import torch
import os
import sys
import matplotlib

from PIL import Image
from dotenv import load_dotenv
from scipy.ndimage import center_of_mass
from src.third_party.depth_anything_v2.dpt import DepthAnythingV2
from ultralytics import YOLO
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

xFormers not available
xFormers not available


### Specify the input image

In [6]:
input_image = "2024_1126_150000F_trimmed_vessel_detected_shot_id_1_frame_79.png" 
input_image_name = input_image.split(".png")[0]

### Object Detection using YOLO

In [11]:
# Load YOLO model (We're using YOLO 11 XL for segmentation model for the masks)
model_name = "yolo11x-seg"
model = YOLO(f"./yolo_weights/{model_name}.pt")

# Run object detection
results = model([f"outputs/2024_1126_150000F_trimmed/tracked_vessels/2024_1126_150000F_trimmed_vessel_detected_shot_id_1_frame_79.png"], imgsz=1920, conf=0.2)

# View results
for r in results:
    r.save(filename=f"_{model_name}_{input_image_name}.jpg")  # save to disk

# Filter for boats
boat_class_id = next(k for k, v in model.names.items() if v.lower() == "boat")

for r in results:
    # Extract masks, class IDs, and confidence scores
    if r.masks is not None and r.boxes is not None:
        masks = r.masks.data.cpu().numpy()  # Shape: (N, H, W)
        class_ids = r.boxes.cls.cpu().numpy()  # Class IDs for masks
        confidences = r.boxes.conf.cpu().numpy()  # Confidence scores for masks

        # Filter indices for the 'boat' class
        boat_indices = np.where(class_ids == boat_class_id)[0]

        if len(boat_indices) > 0:
            # Find the boat with the highest confidence score
            highest_conf_index = boat_indices[np.argmax(confidences[boat_indices])]
            boat_mask = masks[highest_conf_index]  # Binary mask for the boat with highest confidence
            highest_confidence = confidences[highest_conf_index]

            print(f"Highest confidence for a boat: {highest_confidence}")
            break


0: 1088x1920 1 person, 5 boats, 252.4ms
Speed: 0.0ms preprocess, 252.4ms inference, 2.5ms postprocess per image at shape (1, 3, 1088, 1920)
Highest confidence for a boat: 0.8599649667739868


In [53]:
import numpy as np

# Load the saved arrays
boat_mask = np.load('./outputs/2024_1126_143639F_trimmed/object_detection/mask_arrays/2024_1126_143639F_trimmed_mask_array_id_1_frame_355.npy')
depth = np.load('./outputs/2024_1126_143639F_trimmed/depth_estimation/depth_arrays/2024_1126_143639F_trimmed_depth_array_id_1_frame_355.npy')


In [55]:
print(depth.shape)
print(boat_mask.shape)

(1080, 1920)
(1080, 1920)


### Depth Estimation using DepthAnythingV2

In [26]:
# Prereq to download the vitl weights and save it in the checkpoints folder inside the third_party folder
# We're just using the large version

DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}

encoder = 'vitl' # or 'vits', 'vitb', 'vitg'

model = DepthAnythingV2(**model_configs[encoder])
# model.load_state_dict(torch.load(f'checkpoints/depth_anything_v2_{encoder}.pth', map_location='cpu')) Original code but were utilizing depth-anything as a submodule/third-party
model.load_state_dict(torch.load(f'src/third_party/checkpoints/depth_anything_v2_{encoder}.pth', map_location='cpu')) 

# raw_img = cv2.imread(f'data/tracked_vessels/{input_image}')
raw_img = cv2.imread(f'outputs/2024_1126_143639F_trimmed/tracked_vessels/2024_1126_143639F_trimmed_vessel_detected_shot_id_1_frame_355.png')
depth = model.infer_image(raw_img) # HxW raw depth map in numpy

# Save array:
input_image_name = input_image.split(".png")[0]
#np.savetxt(f"{input_image_name}.txt", depth, fmt="%.5f")

### Calculations

In [46]:
# Find the center of the boat
center_y, center_x = center_of_mass(boat_mask)  # Center in (y, x) format

# Measure the height in pixels
rows, _ = np.where(boat_mask > 0)
boat_height_pixels = np.max(rows) - np.min(rows)

print(f"Boat center: ({center_x}, {center_y})")
print(f"Boat height (pixels): {boat_height_pixels}")

Boat center: (1312.809682878052, 623.4211370678856)
Boat height (pixels): 135


In [28]:
# Normalize depth map
depth_normalized = (depth - depth.min()) / (depth.max() - depth.min()) * 255
depth_colored = cv2.applyColorMap(depth_normalized.astype(np.uint8), cv2.COLORMAP_JET)
depth_resized = cv2.resize(depth_colored, (boat_mask.shape[1], boat_mask.shape[0]), interpolation=cv2.INTER_LINEAR)

# Overlay the mask onto the depth map visualization
boat_overlay = depth_resized.copy()
boat_overlay[boat_mask > 0] = [0, 255, 0]  # Green for the boat mask

# Annotate the image with the center and height
cv2.circle(boat_overlay, (int(center_x), int(center_y)), 5, (255, 0, 0), -1)  # Mark center in red
cv2.putText(
    boat_overlay,
    f"Height: {boat_height_pixels}px",
    (int(center_x) - 50, int(center_y) - 20),
    cv2.FONT_HERSHEY_SIMPLEX,
    0.6,
    (255, 255, 255),
    2,
)

array([[[128,   0,   0],
        [128,   0,   0],
        [128,   0,   0],
        ...,
        [128,   0,   0],
        [128,   0,   0],
        [128,   0,   0]],

       [[128,   0,   0],
        [128,   0,   0],
        [128,   0,   0],
        ...,
        [128,   0,   0],
        [128,   0,   0],
        [128,   0,   0]],

       [[128,   0,   0],
        [128,   0,   0],
        [128,   0,   0],
        ...,
        [128,   0,   0],
        [128,   0,   0],
        [128,   0,   0]],

       ...,

       [[  0, 252, 255],
        [  0, 252, 255],
        [  0, 252, 255],
        ...,
        [174, 255,  82],
        [174, 255,  82],
        [174, 255,  82]],

       [[  0, 252, 255],
        [  0, 252, 255],
        [  0, 252, 255],
        ...,
        [174, 255,  82],
        [174, 255,  82],
        [174, 255,  82]],

       [[  0, 252, 255],
        [  0, 252, 255],
        [  0, 252, 255],
        ...,
        [170, 255,  86],
        [170, 255,  86],
        [170, 255,  86]]

In [29]:
# Resize the depth map to match the boat_mask dimensions
depth_resized = cv2.resize(depth_normalized, (boat_mask.shape[1], boat_mask.shape[0]), interpolation=cv2.INTER_LINEAR)

# Convert boat_mask to boolean
boat_mask_boolean = boat_mask > 0  # Ensure the mask is boolean (True/False)

# Calculate rows_with_mask using the original mask
rows_with_mask = np.any(boat_mask_boolean, axis=1)

# Get the depth values corresponding to the mask
boat_depth_values = depth_resized[rows_with_mask, :]
boat_depth_average = boat_depth_values[boat_mask_boolean[rows_with_mask, :]].mean()

# Get the depth at the boat center
boat_center_depth = depth_resized[int(center_y), int(center_x)]

print(f"Depth at boat center: {boat_center_depth}")
print(f"Average boat depth: {boat_depth_average}")


Depth at boat center: 36.05406188964844
Average boat depth: 33.786720275878906


In [30]:
# Save the combined visualization
cv2.imwrite(f"{input_image}.png", boat_overlay)

True

In [31]:
print(f"YOLO processed mask shape: {boat_mask.shape}")  # e.g., (1216, 2112)
print(f"Original image shape: {depth.shape}")  # e.g., (1183, 2100)

YOLO processed mask shape: (1080, 1920)
Original image shape: (1080, 1920)


In [32]:
# Get the depth at the center of the boat
boat_center_depth = depth_resized[int(center_y), int(center_x)]
print(f"Depth at boat center: {boat_center_depth}")

Depth at boat center: 36.05406188964844


In [50]:
# Define the top-left and bottom-right coordinates of the rectangular area
rect_top_left_x = 1085 # X-coordinate of the top-left corner
rect_top_left_y = 637  # Y-coordinate of the top-left corner
rect_bottom_right_x = 1097   # X-coordinate of the bottom-right corner
rect_bottom_right_y = 708  # Y-coordinate of the bottom-right corner

# Extract the depth values within the rectangle
rect_depth_values = depth_resized[
    rect_top_left_y:rect_bottom_right_y,
    rect_top_left_x:rect_bottom_right_x
]

# Calculate the average depth within the rectangle
rect_depth_average = rect_depth_values.max()

# Print the average depth
print(f"Average depth in the rectangle: {rect_depth_average}")

Average depth in the rectangle: 227.80648803710938


In [41]:
h_orange_rect_px = 71
h_meter_stick_px = 200
h_meter_stick_actual = 1
h_orange_rect_actual = h_orange_rect_px/h_meter_stick_px

In [42]:
h_orange_rect_actual

0.355

In [45]:
# OG ( ORANGE RECTANGLE )
135*(0.355/71)*(227.80648803710938/33.786720275878906)

4.551178041830483

In [38]:
# JUST RULER
128*(1/598)*(28.99/31.47)

0.1971788176455147